In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/amir/projects/frame_stack


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as jn
import os
import yaml
import torch
from tqdm.notebook import tqdm


In [3]:
with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)
with open('pathes.yaml') as conf_file:
    path_config = yaml.safe_load(conf_file)

pressure_path = path_config['p_video_path']
signal_path = path_config['test_s_video_path']


In [4]:
print(os.listdir(pressure_path))

['1222', 's_440', '531', '34', 's_540', '37', 'test', 's_62', 's_201', 's_269', 's_110', '1743', 's_67', '1744', '62', 's_332', '540', 's_502', 's_327', 's_1600', '1386', '178', '1610', 's_125', 's_1744', '71', '155', 's_353', '488', 's_731', '237', 's_329', 's_494', '869', 's_1218', 's_488', 's_1601', 's_34', '125', 's_212', 's_504', 's_537', '338', '743', 's_95', '360', 's_265', 's_328', '332', 's_337', 's_546', 's_1612', '67', '502', '1246', '1791', 's_557', 's_1629', '235', 's_1244', 's_237', 's_1181', 's_104', 's_1222', 's_527', 's_1613', '1601', '1300', '1606', 's_155', '95', 's_1639', '104', '201', 's_35', '337', 's_1225', '265', '212', '269', 's_208', 's_1386', 's_360', 's_747', '1244', 's_235', 's_536', '1745', '58', 's_324', 's_60', '747', '731', '327', '840', 's_336', '336', 's_840', '353', '459', 's_743', '704', '500', 's_1791', '50', '1225', '527', 's_759', 's_614', 's_381', 's_365', 's_1598', '530', 's_1599', '494', '1635', 's_745', 's_704', '742', 's_71', 's_1597', 's_53

In [4]:
val_data = []
search_path = jn(pressure_path, '1222')
for folder in os.listdir(search_path):
    val_data.append(np.expand_dims(np.load(jn(search_path, folder, "prepared.npz"))["arr_0"], 0))
print(len(val_data))
print(val_data[0].shape)
val_data = np.concatenate(val_data)
print(val_data.shape)

64
(1, 300, 64, 64)
(64, 300, 64, 64)


In [6]:
val_data.dtype

dtype('float32')

In [5]:
import pandas as pd

In [9]:
if os.path.exists('data/dataset_test_table.csv'):
    pivot_table = pd.read_csv('data/dataset_test_table.csv')
else:
    pivot_table = pd.DataFrame()

test = pd.DataFrame({'name': 'many npz', 
                     "floats_number": val_data.size, 
                     'save_time': 0,
                     'load_time': 0,
                     "save_size": 0,
                     }, index=[0])
pivot_table

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.850,1.090,78643200,npz one
1,3.4,1.890,0.787,78643200,npz many
2,301.0,3.670,0.085,78643200,npy many
3,301.0,4.340,0.201,78643200,hd5 uncomp
4,3.6,2.000,1.140,78643200,hd5 gzip
5,6.9,0.751,0.846,78643200,hd5 lzf
6,68.0,3.630,2.670,78643200,hd5 szip
7,6.5,0.474,0.382,78643200,hd5 Blosc
8,6.5,0.460,0.400,78643200,hd5 Blosc


In [10]:
test.insert(0, "test_col", 0)
test

,test_col,name,floats_number,load_time,save_time,save_size
0,0,many npz,78643200,0,0,0


In [7]:
save_folder = 'test_folder'
os.makedirs(save_folder)

In [8]:
import glob

## Numpy

npz one

In [10]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [11]:
%%timeit
# saving
np.savez_compressed(jn(save_folder, 'test.npz'), val_data)

2 s ± 20.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
# loading
np.load(jn(save_folder, 'test.npz'))['arr_0']

938 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
!du -sh test_folder

3.3M	test_folder


In [20]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'npz one')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 2.05)
log.insert(0, 'load_time, s', 2.18)
log.insert(0, 'save_size, MB', 3.3)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,2.05,2.18,78643200,npz one


npz many

In [13]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [14]:
%%timeit
# saving
for i, batch in enumerate(val_data):
    np.savez_compressed(jn(save_folder, f'test{i}.npz'), batch)

1.95 s ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
# loading
for file in glob.glob(jn(save_folder, "*")):
    np.load(file)['arr_0']

857 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
!du -sh test_folder

3.4M	test_folder


In [25]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'npz many')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 2.52)
log.insert(0, 'load_time, s', 1.24)
log.insert(0, 'save_size, MB', 3.4)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,2.05,2.18,78643200,npz one
0,3.4,2.52,1.24,78643200,npz many


npy

In [26]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [27]:
%%timeit
# saving
for i, batch in enumerate(val_data):
    np.save(jn(save_folder, f'test{i}.npy'), batch)

5.01 s ± 429 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
# loading
for file in glob.glob(jn(save_folder, "*")):
    np.load(file)

185 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
!du -sh test_folder

301M	test_folder


In [30]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'npy many')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 5)
log.insert(0, 'load_time, s', 0.185)
log.insert(0, 'save_size, MB', 301)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,2.05,2.180,78643200,npz one
0,3.4,2.52,1.240,78643200,npz many
0,301.0,5.00,0.185,78643200,npy many


# HDF5

In [28]:
import h5py

In [29]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [30]:
%%timeit
# saving

with h5py.File(jn(save_folder, 'random.hdf5'), 'w') as f:
    dset = f.create_dataset("default", data=val_data)

4.34 s ± 645 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%%timeit
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with h5py.File(file, 'r') as f:
       data = f['default'][:]

202 ms ± 74.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
!du -sh test_folder

301M	test_folder


In [46]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'hd5 uncomp')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 4.34)
log.insert(0, 'load_time, s', 0.201)
log.insert(0, 'save_size, MB', 301)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.85,1.090,78643200,npz one
0,3.4,1.89,0.787,78643200,npz many
0,301.0,3.67,0.085,78643200,npy many
0,301.0,4.34,0.201,78643200,hd5 uncomp


## hdf5 with gzip

In [33]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [34]:
%%timeit
# saving

with h5py.File(jn(save_folder, 'random.hdf5'), 'w') as f:
    dset = f.create_dataset("default", data=val_data, compression="gzip")

2.25 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%%timeit
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with h5py.File(file, 'r') as f:
       data = f['default'][:]

1.25 s ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
!du -sh test_folder

3.6M	test_folder


In [47]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'hd5 gzip')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 2)
log.insert(0, 'load_time, s', 1.14)
log.insert(0, 'save_size, MB', 3.6)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.85,1.090,78643200,npz one
0,3.4,1.89,0.787,78643200,npz many
0,301.0,3.67,0.085,78643200,npy many
0,301.0,4.34,0.201,78643200,hd5 uncomp
0,3.6,2.00,1.140,78643200,hd5 gzip


## hdf5 with LZF

In [37]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [38]:
%%timeit
# saving

with h5py.File(jn(save_folder, 'random.hdf5'), 'w') as f:
    dset = f.create_dataset("default", data=val_data, compression="lzf")

705 ms ± 68.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with h5py.File(file, 'r') as f:
       data = f['default'][:]

874 ms ± 79.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
!du -sh test_folder

6.9M	test_folder


In [48]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'hd5 lzf')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 0.751)
log.insert(0, 'load_time, s', 0.846)
log.insert(0, 'save_size, MB', 6.9)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.4,1.890,0.787,78643200,npz many
0,301.0,3.670,0.085,78643200,npy many
0,301.0,4.340,0.201,78643200,hd5 uncomp
0,3.6,2.000,1.140,78643200,hd5 gzip
0,6.9,0.751,0.846,78643200,hd5 lzf


## hdf5 with szip

In [41]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [42]:
%%timeit
# saving

with h5py.File(jn(save_folder, 'random.hdf5'), 'w') as f:
    dset = f.create_dataset("default", data=val_data, compression="szip")

3.63 s ± 833 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with h5py.File(file, 'r') as f:
       data = f['default'][:]

2.67 s ± 540 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
!du -sh test_folder

68M	test_folder


In [49]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'hd5 szip')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 3.63)
log.insert(0, 'load_time, s', 2.67)
log.insert(0, 'save_size, MB', 68)
pivot_table = pd.concat([pivot_table, log])
pivot_table.tail()

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,301.0,3.670,0.085,78643200,npy many
0,301.0,4.340,0.201,78643200,hd5 uncomp
0,3.6,2.000,1.140,78643200,hd5 gzip
0,6.9,0.751,0.846,78643200,hd5 lzf
0,68.0,3.630,2.670,78643200,hd5 szip


## hdf5 with Blosc

In [50]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [51]:
import hdf5plugin

In [59]:
%%timeit -n 5
# saving

with h5py.File(jn(save_folder, 'random.hdf5'), 'w') as f:
    dset = f.create_dataset("default", data=val_data, 
                            **hdf5plugin.Blosc(cname='blosclz', clevel=4, shuffle=hdf5plugin.Blosc.SHUFFLE))

466 ms ± 41.5 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [60]:
%%timeit -n 5
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with h5py.File(file, 'r') as f:
       data = f['default'][:]

378 ms ± 37.2 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [61]:
!du -sh test_folder

6.5M	test_folder


In [58]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'hd5 Blosc cmp=4')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 0.46)
log.insert(0, 'load_time, s', 0.40)
log.insert(0, 'save_size, MB', 6.5)
pivot_table = pd.concat([pivot_table, log])
pivot_table

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.850,1.090,78643200,npz one
0,3.4,1.890,0.787,78643200,npz many
0,301.0,3.670,0.085,78643200,npy many
0,301.0,4.340,0.201,78643200,hd5 uncomp
0,3.6,2.000,1.140,78643200,hd5 gzip
0,6.9,0.751,0.846,78643200,hd5 lzf
0,68.0,3.630,2.670,78643200,hd5 szip
0,6.5,0.474,0.382,78643200,hd5 Blosc
0,6.5,0.460,0.400,78643200,hd5 Blosc


## Pytables

In [30]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [17]:
import tables as tb


In [37]:
%%timeit -n 5
# saving
 
for file in glob.glob(jn(save_folder, "*")): os.remove(file)
with tb.open_file(jn(save_folder, 'random.h5'), mode = "w") as fileh:
    root = fileh.root
    fileh.create_array(root, "array1", val_data)
    fileh.close()

311 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [38]:
%%timeit -n 5
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with tb.open_file(file, mode='r') as f:
       data = f.root["array1"][:]

158 ms ± 9.61 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [39]:
!du -sh test_folder

301M	test_folder


In [40]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'pytab without comp')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 0.3)
log.insert(0, 'load_time, s', 0.15)
log.insert(0, 'save_size, MB', 301)
pivot_table = pd.concat([pivot_table, log])
pivot_table

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.850,1.090,78643200,npz one
1,3.4,1.890,0.787,78643200,npz many
2,301.0,3.670,0.085,78643200,npy many
3,301.0,4.340,0.201,78643200,hd5 uncomp
4,3.6,2.000,1.140,78643200,hd5 gzip
5,6.9,0.751,0.846,78643200,hd5 lzf
6,68.0,3.630,2.670,78643200,hd5 szip
7,6.5,0.474,0.382,78643200,hd5 Blosc
8,6.5,0.460,0.400,78643200,hd5 Blosc
0,301.0,0.300,0.150,78643200,pytab without comp


## Pytables compressed lzo

In [41]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [42]:
import tables as tb


In [46]:
%%timeit -n 5
# saving
 
with tb.open_file(jn(save_folder, 'random.h5'), mode = "w") as fileh:
    root = fileh.root
    filters = tb.Filters(complevel=4, complib='lzo')

    arr = fileh.create_carray(root, "array1", obj=val_data, filters=filters)
    fileh.close()


441 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [47]:
%%timeit -n 5
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with tb.open_file(file, mode='r') as f:
       data = f.root["array1"][:]

538 ms ± 6.35 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [45]:
!du -sh test_folder

6.1M	test_folder


In [48]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'pytab with lzo')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 0.44)
log.insert(0, 'load_time, s', 0.54)
log.insert(0, 'save_size, MB', 6)
pivot_table = pd.concat([pivot_table, log])
pivot_table

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.850,1.090,78643200,npz one
1,3.4,1.890,0.787,78643200,npz many
2,301.0,3.670,0.085,78643200,npy many
3,301.0,4.340,0.201,78643200,hd5 uncomp
4,3.6,2.000,1.140,78643200,hd5 gzip
5,6.9,0.751,0.846,78643200,hd5 lzf
6,68.0,3.630,2.670,78643200,hd5 szip
7,6.5,0.474,0.382,78643200,hd5 Blosc
8,6.5,0.460,0.400,78643200,hd5 Blosc
0,301.0,0.300,0.150,78643200,pytab without comp


## Pytables compressed blosc

In [49]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [50]:
import tables as tb


In [51]:
%%timeit -n 7
# saving

with tb.open_file(jn(save_folder, 'random.h5'), mode = "w") as fileh:
    root = fileh.root
    filters = tb.Filters(complevel=4, complib='blosc')

    arr = fileh.create_carray(root, "array1", obj=val_data, atom=tb.Float32Atom(), shape=val_data.shape, filters=filters)
    fileh.close()




455 ms ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 7 loops each)


In [52]:
%%timeit -n 7
# loading
for file in glob.glob(jn(save_folder, "*")):
    
    with tb.open_file(file, mode='r') as f:
       data = f.root["array1"][:]

384 ms ± 6.38 ms per loop (mean ± std. dev. of 7 runs, 7 loops each)


In [53]:
!du -sh test_folder

6.0M	test_folder


In [54]:
log = pd.DataFrame(index=[0])
log.insert(0, 'name', 'pytab with blosc')
log.insert(0, 'floats_number', val_data.size)
log.insert(0, 'save_time, s', 0.45)
log.insert(0, 'load_time, s', 0.38)
log.insert(0, 'save_size, MB', 6)
pivot_table = pd.concat([pivot_table, log])
pivot_table

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.850,1.090,78643200,npz one
1,3.4,1.890,0.787,78643200,npz many
2,301.0,3.670,0.085,78643200,npy many
3,301.0,4.340,0.201,78643200,hd5 uncomp
4,3.6,2.000,1.140,78643200,hd5 gzip
5,6.9,0.751,0.846,78643200,hd5 lzf
6,68.0,3.630,2.670,78643200,hd5 szip
7,6.5,0.474,0.382,78643200,hd5 Blosc
8,6.5,0.460,0.400,78643200,hd5 Blosc
0,301.0,0.300,0.150,78643200,pytab without comp


### Uint8 comparings

In [81]:
from PIL import Image

In [83]:
val_data.shape

(64, 300, 64, 64)

## gif

In [92]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [94]:
chains = val_data[0]
# norming
chains = ((chains - chains.min()) * 255 /
          (chains.max() - chains.min())).astype(np.uint8)
images = []
for i in range(len(chains)):
    images.append(Image.fromarray(chains[i]))
images[0].save(jn(save_folder, 'test.gif'),
               save_all=True,
               loop=0,
               duration=30,
               append_images=images[1:])

In [95]:
!du -sh test_folder

12K	test_folder


## numpy

In [100]:
for file in glob.glob(jn(save_folder, "*")): os.remove(file)

In [101]:
chains = val_data[0]
# norming
chains = ((chains - chains.min()) * 255 /
          (chains.max() - chains.min())).astype(np.uint8)
images = []
for i in range(len(chains)):
    images.append(Image.fromarray(chains[i]))
np.savez_compressed(jn(save_folder, 'test.npy'), chains)


In [102]:
!du -sh test_folder

8.0K	test_folder


In [55]:
pivot_table.to_csv('data/dataset_test_table.csv', index=False)

In [56]:
pivot_table = pd.read_csv('data/dataset_test_table.csv')
pivot_table

,"save_size, MB","save_time, s","load_time, s",floats_number,name
0,3.3,1.850,1.090,78643200,npz one
1,3.4,1.890,0.787,78643200,npz many
2,301.0,3.670,0.085,78643200,npy many
3,301.0,4.340,0.201,78643200,hd5 uncomp
4,3.6,2.000,1.140,78643200,hd5 gzip
5,6.9,0.751,0.846,78643200,hd5 lzf
6,68.0,3.630,2.670,78643200,hd5 szip
7,6.5,0.474,0.382,78643200,hd5 Blosc
8,6.5,0.460,0.400,78643200,hd5 Blosc
9,301.0,0.300,0.150,78643200,pytab without comp


In [83]:
pivot_table = pivot_table[:-1]